In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import gc
from matplotlib import gridspec
import time
import tensorflow as tf

import os
os.environ['CUDA_VISIBLE_DEVICES']="0" 

physical_devices = tf.config.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [2]:
from matplotlib import rc
rc('text', usetex=True)

import matplotlib as mpl
rc('font', family='serif')
rc('text', usetex=True)
rc('font', size=22)
rc('xtick', labelsize=15)
rc('ytick', labelsize=15)
rc('legend', fontsize=15)

mpl.rcParams.update({'font.size': 19})
mpl.rcParams.update({'xtick.labelsize': 18}) 
mpl.rcParams.update({'ytick.labelsize': 18}) 
mpl.rcParams.update({'text.usetex' : False})
mpl.rcParams.update({'axes.labelsize': 18}) 
mpl.rcParams.update({'legend.frameon': False}) 

import matplotlib.pyplot as plt
import mplhep as hep
hep.set_style("CMS") 

<ipython-input-2-99a23e5daf14>:21: FutureWarning: ``set_style`` is deprecated: Naming convention is changing to match mpl. Use ``mplhep.style.use()``.
  hep.set_style("CMS")


In [3]:
bins = {}

#jet pt
bins['jetpt'] = np.logspace(np.log10(10),np.log10(100),7)

#jet eta
bins['jeteta'] = np.linspace(-1,2.5,6)

#dphi
bins['dphi'] = np.logspace(np.log10(0.03),np.log10(np.pi/2.0),9) - 0.03
bins['dphi'] = bins['dphi'][1:]
bins['dphi'][0] = 0.0

#qt
bins['qt'] = np.logspace(np.log10(0.03),np.log10(3.03),9) - 0.03
bins['qt'] = bins['qt'][1:]
bins['qt'][0] = 0.0

bins['Q2'] = np.logspace(np.log10(100),np.log10(100000),5)

In [ ]:
#Read in the MC
mc = pd.read_pickle("/clusterfs/ml4hep/yxu2/unfolding_mc_inputs/Rapgap_nominal.pkl")


theta0_G = mc[['gene_px','gene_py','gene_pz','genjet_pt','genjet_eta','genjet_phi','genjet_dphi','genjet_qtnorm', 'gen_Q2']].to_numpy()


weights_MC_sim = mc['wgt']
pass_reco = np.array(mc['pass_reco'])
pass_truth = np.array(mc['pass_truth'])
pass_fiducial = np.array(mc['pass_fiducial'])

Q2 = theta0_G[:,8]
Q2 = Q2[pass_fiducial==1]
theta0_G = theta0_G[:,0:8]
#pass_truth = pass_truth[pass_fiducial==1]
#pass_reco = pass_reco[pass_fiducial==1][pass_truth==1]

In [ ]:
n_MC,_,_=plt.hist(Q2,bins=bins["Q2"],weights=weights_MC_sim[pass_fiducial==1],density=True,histtype="step",color="black",label="MC")
np.save("outputfiles/MCCrossSection_Q2_Rapgap_nominal_ep0607_data_nominal.npy", n_MC)



In [ ]:
obs_idx = {}
obs_idx['dphi'] = 6
obs_idx['jetpt'] = 3
obs_idx['qt'] = 7
obs_idx['jeteta'] = 4

cuts = [150,200,250,48000]

In [ ]:
for k in range(len(cuts)-1):
    Q2_cut = np.where((cuts[k] < Q2) & (Q2 < cuts[k+1]), True, False)
    cut_label = str(cuts[k])+" < Q2 < "+str(cuts[k+1]) #+ " (GeV^2)"
    
    for observable in ['dphi','jetpt','qt','jeteta']:
        n_MC,_,_=plt.hist(theta0_G[pass_fiducial==1][:,obs_idx[observable]][Q2_cut],bins=bins[observable],weights=weights_MC_sim[pass_fiducial==1][Q2_cut],density=True,histtype="step",color="black",label="MC")
        np.save("outputfiles/MCCrossSection_"+observable+"_Django_nominal_ep0607_data_nominal_"+cut_label+".npy", n_MC)
        
